In [54]:
import plotly.graph_objects as go
import numpy as np
from pymongo import MongoClient
from bson import ObjectId
import logging
import random
import pandas as pd

In [55]:
def seismic_ascii_parser(file_path):
    all_data = []
    head = ""
    xarray = []
    yarray = []
    zarray = []
    with open(file_path, 'r') as f:
        content = f.readlines()
        for line in content:
            if '#' in line:
                head += line
                continue
            l = line.split(' ')
            xarray.append(float(l[0]))
            yarray.append(float(l[1]))
            zarray.append(float(l[2]))

    df = pd.DataFrame({'x': xarray, 'y': yarray, 'z': zarray})
    all_data.append({"test": df})
    return create_seismic_ascii_date(all_data)[0]

def create_seismic_ascii_date(data_frame):
    for dataframe_data in data_frame:
        _, value = next(iter(dataframe_data.items()))
        return value.to_dict("records"),

In [56]:
def extract_unique_coordinates(coord_list):

    x_set = set()
    y_set = set()

    for coord in coord_list:
        x_set.add(coord['x'])
        y_set.add(coord['y'])

    x_unique = list(x_set)
    y_unique = list(y_set)

    return x_unique, y_unique



In [57]:
TWT_Bottom = seismic_ascii_parser(r"C:/HV/Seismic/datas/TWT_Bottom_U1.txt")
TWT_Top = seismic_ascii_parser(r"C:/HV/Seismic/datas/TWT_Top_U1.txt")

In [58]:
x_coords, y_coords = extract_unique_coordinates(TWT_Top)

print(len(x_coords))
print(len(y_coords))
print(min(x_coords), max(x_coords), (max(x_coords) - min(x_coords)) / 186)
print(min(y_coords), max(y_coords), (max(y_coords) - min(y_coords)) / 214)


286
280
445946.632991 460196.632991 76.61290322580645
7004198.214381 7018148.214381 65.18691588785046


In [59]:
start_x = min(x_coords)
stop_x = max(x_coords)
start_y = min(y_coords)
stop_y = max(y_coords)
delta_x = (max(x_coords) - min(x_coords)) / 186
delta_y = (max(y_coords) - min(y_coords)) / 214

In [60]:
"""
Создаем свой грид в виде двух областей [x, y, z_min, z_max] размерности 186х214 ячеек над кровлей и подошвой
"""
grid = []
for i in range(186):
    #grid.append([])
    for j in range(214):
        grid.append([start_x + i*delta_x, start_y + j*delta_y, -2350 + random.uniform(-2, 2), -2280 + random.uniform(-2, 2)])



In [61]:
print(len(grid))

39804


In [62]:
df = pd.DataFrame(np.array(grid), columns=['x', 'y', 'z_min', 'z_max'])

In [63]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


# Создание фигуры
fig = go.Figure()

data1 = TWT_Top
data2 = TWT_Bottom
x1 = [point['x'] for point in data1]
y1 = [point['y'] for point in data1]
z1 = [point['z'] for point in data1]
    
x2 = [point['x'] for point in data2]
y2 = [point['y'] for point in data2]
z2 = [point['z'] for point in data2]
 
 
fig.add_trace(go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Top_TWT'
))
    
fig.add_trace(go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(size=3, color='red'),
    name='Bottom_TWT'
))



# Добавление точек для z_min
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z_min'],
    mode='markers',
    name='z_min',
    marker=dict(color='white', size=5)
))

# Добавление точек для z_max
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z_max'],
    mode='markers',
    name='z_max',
    marker=dict(color='yellow', size=5)
))

# Настройка внешнего вида графика
fig.update_layout(
    title='Визуализация областей z_min и z_max',
    #width=800,  # Установка ширины графика
    #height=600,  # Установка высоты графика
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# Показать график
fig.show()
